# Data processing 

## Imports

In [61]:
import numpy as np
import json
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.feature_extraction.text import CountVectorizer

## Load dataset

In [62]:
with open('intents.json', 'r') as f:
    intents = json.load(f)

In [63]:
xy = []
labels = []

for intent in intents['intents']:
    tag = intent['tag']
    labels.append(tag)
    for pattern in intent['patterns']:
        xy.append((pattern, tag))

In [64]:
y_train = []
for (pattern_sentence, tag) in xy:
    label = labels.index(tag)
    y_train.append(label)

y_train = np.array(y_train)

In [65]:
class Sequences(Dataset):
    def __init__(self):
        self.vectorizer = CountVectorizer(stop_words='english', max_df=0.99, min_df=0.005)
        self.sequences = self.vectorizer.fit_transform(sentences)
        self.labels = y_train
        self.token2idx = self.vectorizer.vocabulary_
        self.idx2token = {idx: token for token, idx in self.token2idx.items()}
        
    def __getitem__(self, i):
        return self.sequences[i, :].toarray(), self.labels[i]
    
    def __len__(self):
        return self.sequences.shape[0]

# Model definition

## Hyperparameters

In [72]:
num_epochs = 1000
batch_size = 8
learning_rate = 0.001
hidden_size = 8
input_size = len(dataset.token2idx)
output_size = len(list(set(dataset.labels)))

## Model

In [67]:
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.l1 = nn.Linear(input_size, hidden_size)
        self.l2 = nn.Linear(hidden_size, hidden_size)
        self.l3 = nn.Linear(hidden_size, num_classes)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        out = self.relu(out)
        out = self.l3(out)
        # no activation and no softmax at the end
        return out

# Training

In [68]:
dataset = Sequences()
train_loader = DataLoader(dataset, batch_size=batch_size)

In [69]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = NeuralNet(input_size, hidden_size, output_size).to(device)

In [70]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
for epoch in range(num_epochs):
    losses = []
    total = 0
    for words, labels in train_loader:
        words = words.to(dtype=torch.float).to(device)
        labels = labels.to(dtype=torch.long).to(device)
        
        # Forward pass
        outputs = model(words)
        loss = criterion(outputs.squeeze(), labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    if (epoch+1) % 100 == 0:
        print (f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')
        
print(f'final loss: {loss.item():.4f}')

Epoch [100/1000], Loss: 0.0000
Epoch [200/1000], Loss: 0.0000
Epoch [300/1000], Loss: 0.0000
